## 1. PureActionState

### Example: Create GENERATE state using PureActionState

```lua
stateDiagram-v2
    INIT --> GENERATE
    GENERATE --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> GENERATE
    GENERATE --> FINAL
```


In [6]:
from gai.llm.openai import AsyncOpenAI
from gai.lib.config import config_helper
llm_config = config_helper.get_client_config("ttt")
client = AsyncOpenAI(client_config=llm_config)
response = await client.chat.completions.create(
    messages=[{"role":"user","content":"Tell me a one sentence story."}],
    max_tokens=50,
    stream=True,
)
async for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)


As the school day ended, a shy girl named Sarah nervously approached her crush, Alex, and accidentally spilled juice all over his shirt, prompting them to quickly and awkwardly clean it together in the bathroom.

In [9]:
from gai.llm.openai import AsyncOpenAI
from gai.lib.config import config_helper
llm_config = config_helper.get_client_config("ollama")
client = AsyncOpenAI(client_config=llm_config)
response = await client.chat.completions.create(
    model="llama3.2:3b",
    messages=[{"role":"user","content":"Tell me a one sentence story."}],
    max_tokens=50,
    stream=True,
)
async for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)


As she opened the small, dusty trunk in the attic, Emily's grandmother's long-forgotten locket fell out, revealing a photograph of her own face staring back at her from a life she had never known existed.

In [3]:
from gai.asm import AgenticStateMachine
from gai.lib.config import config_helper
llm_config = config_helper.get_client_config("ttt")


async def generate_action(state):
    from openai import AsyncOpenAI

    client = AsyncOpenAI(llm_config=llm_config)

    # Import data from state_bag
    agent_name = state.machine.state_bag.get("name", "Assistant")
    user_message = state.machine.state_bag.get(
        "user_message",
        "If you are seeing this, that means I have forgotten to add a user message. Remind me.",
    )

    # Execute

    state.machine.monologue.add_user_message(
        content=f"Your name is {agent_name}. You are a helpful assistant. {agent_name}, {user_message}"
    )

    from gai.messages import message_helper

    chat_messages = state.machine.monologue.list_chat_messages()
    chat_messages = message_helper.shrink_messages(chat_messages)


    async def streamer():
        response = await client.chat.completions.create(
            messages=chat_messages,
            max_tokens=50,
            stream=True,
        )
        content = ""
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk, str) and chunk:
                content += chunk
                yield chunk
        state.machine.monologue.add_assistant_message(content=content)

    state.machine.state_bag["streamer"] = streamer()


## Step 1: INIT

with AgenticStateMachine.StateMachineBuilder("""
    INIT --> GENERATE
    GENERATE --> FINAL
    """) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {},
            },
            "GENERATE": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "GENERATE",
                "action": "generate",
                "output_data": ["streamer"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        user_message="Hello, world!",
        generate=generate_action,
    )
fsm.restart()

## Step 2: INIT --> GENERATE

# Fixed: fsm.run_async() returns a boolean, get streamer from state bag
await fsm.run_async()
streamer = fsm.state_bag.get("streamer")
if streamer:
    async for chunk in streamer:
        print(chunk, end="", flush=True)
print("\n\n")

## Step 3: GENERATE --> FINAL
await fsm.run_async()

## Step 4: Print the state history
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)

TypeError: AsyncOpenAI.__init__() got an unexpected keyword argument 'llm_config'

### Example: Create conditional state using PurePredicateState

In this example, we will demonstrate a **Predicate** state whose sole existence is to decide if the output is **true** or **false**


```mermaid
stateDiagram-v2
INIT --> PREDICATE: next / action
PREDICATE --> TRUE(): next / action / condition_true
    TRUE() --> FINAL: next / action
PREDICATE --> FALSE(): next / action / condition_false
    FALSE() --> FINAL: next / action
```

Ask if the sky is blue

In [8]:
from gai.asm import AgenticStateMachine

# Define the predicate: simplified for testing
def sky_is_blue(state) -> bool:
    # Simplified - just return True for testing
    return True


# Define the predicate actions
async def run_if_true(state):
    print("Action: THE ANSWER IS TRUE")


async def run_if_false(state):
    print("Action: THE ANSWER IS FALSE")


with AgenticStateMachine.StateMachineBuilder("""
    INIT --> PREDICATE
    PREDICATE --> TRUE: condition_true
        TRUE --> FINAL 
    PREDICATE --> FALSE: condition_false
        FALSE --> FINAL    
    """) as builder:
    fsm = builder.build(
        {
            "PREDICATE": {
                "module_path": "gai.asm.states",
                "class_name": "PurePredicateState",
                "title": "PREDICATE",
                "predicate": "sky_is_blue",
                "output_data": ["predicate_result"],
                "conditions": ["condition_true", "condition_false"],
            },
            "TRUE": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "input_data": {"title": "TRUE"},
                "action": "run_if_true",
            },
            "FALSE": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "input_data": {"title": "FALSE"},
                "action": "run_if_false",
            },
        },
        sky_is_blue=sky_is_blue,
        run_if_true=run_if_true,
        run_if_false=run_if_false,
    )

In [9]:
fsm.restart()

# First run: INIT -> PREDICATE (evaluates predicate) 
await fsm.run_async()

# state output:
print("Current state is", fsm.state)
print("Result is", fsm.state_bag["predicate_result"])

Current state is PREDICATE
Result is True


In [10]:
# Second run: PREDICATE -> TRUE (based on condition)
await fsm.run_async()

# state output:
print("Current state is", fsm.state)

# Third run: TRUE -> FINAL
await fsm.run_async()

# state output:
print("Current state is", fsm.state)

Action: THE ANSWER IS TRUE
Current state is TRUE
Current state is FINAL


In [11]:
from gai.asm import AgenticStateMachine


async def generate_action(state):
    from openai import AsyncOpenAI

    client = AsyncOpenAI()

    # Import data from state_bag
    agent_name = state.machine.state_bag.get("name", "Assistant")
    user_message = state.machine.state_bag.get(
        "user_message",
        "If you are seeing this, that means I have forgotten to add a user message. Remind me.",
    )

    from openai import AsyncOpenAI

    client = AsyncOpenAI()

    # Execute

    state.machine.monologue.add_user_message(
        content=f"Your name is {agent_name}. You are a helpful assistant.{agent_name}, {user_message}"
    )

    from gai.messages import message_helper

    chat_messages = state.machine.monologue.list_chat_messages()
    chat_messages = message_helper.shrink_messages(chat_messages)


    async def streamer():
        response = await client.chat.completions.create(
            model="gpt-4o-mini",  # Fixed: use valid model name
            messages=chat_messages,
            max_tokens=50,
            stream=True,
        )
        content = ""
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk, str) and chunk:
                content += chunk
                yield chunk
        state.machine.monologue.add_assistant_message(content=content)

    state.machine.state_bag["streamer"] = streamer()


## Step 1: INIT

with AgenticStateMachine.StateMachineBuilder("""
    INIT --> GENERATE
    GENERATE --> FINAL
    """) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {},
            },
            "GENERATE": {
                "module_path": "gai.asm.states",
                "class_name": "PureActionState",
                "title": "GENERATE",
                "action": "generate",
                "output_data": ["streamer"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        user_message="Hello, world!",
        generate=generate_action,
    )
fsm.restart()

In [12]:
## Step 2: INIT --> GENERATE

# Fixed: fsm.run_async() returns a boolean, get streamer from state bag
await fsm.run_async()
streamer = fsm.state_bag.get("streamer")
if streamer:
    async for chunk in streamer:
        print(chunk, end="", flush=True)
print("\n\n")

Hello! How can I assist you today?


